In [ ]:
%matplotlib inline
from ipyparallel import Client, error
cluster=Client(profile="mpi")
view=cluster[:]
view.block=True

try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Connections involving distributed variables

The purpose of this section is to describe the behavior you should expect when connecting two variables together when at least one of them is a distributed variable.

## Distributed output to non-distributed input
Because distributed variables may have different sizes and/or values on different ranks, and non-distributed variables have the same value and size on all ranks, the only way that a distributed to non-distributed connection is allowed is if `src_indices` are specified for the non-distributed input and that the specified `src_indices` are identical on all ranks where the input variable exists.  Otherwise the non-distributed inputs could have different values on different ranks, which is illegal. Note that in previous versions of OpenMDAO, the behavior when connecting a distributed output to a non-distributed input when `src_indices` were not specified was that the non-distributed input on each rank would be the same size as the full distributed output.  This is no longer the case.  In fact, this case is no longer allowed.  However, you can still achieve the same behavior by specifying `src_indices` of `om.slicer[:]` on the input, which explicitly specifies that the input on each rank is the same size as the full distributed output.  For example:

```python
group.connect('mydistcomp.out', 'mynondistributed.in', src_indices=om.slicer[:])
```

## Distributed output to distributed input
When connecting two distributed variables, you can specify `src_indices` on the input however you choose, or OpenMDAO will automatically assign `src_indices` to the input based on the size of the input in each rank.  For example, if the component that owns the input is running on 3 ranks with input sizes of [2, 3, 4], then `src_indices` of [0, 1], [2, 3, 4], and [5, 6, 7, 8] will be specified on the 3 ranks.  If, however, the global size of the output does not equal the global size of the input, an exception will be raised saying that OpenMDAO is not able to determine the `src_indices`.

## Non-distributed output to distributed input
These types of connections are deprecated and will become errors in a future release, but for now, you are allowed to connect a non-distributed output to a distributed input, provided that the global sizes of input and output are equal and the inputs are the same size on every rank.

## Example
The following example shows the `MixedDistrib2` component, which contains a non-distributed input, a distributed input, and a distributed output. We show how to connect the non-distributed input and the distributed input to a distributed output.

In [ ]:
%%px

import numpy as np

import openmdao.api as om
from openmdao.utils.array_utils import evenly_distrib_idxs
from openmdao.utils.mpi import MPI


class MixedDistrib2(om.ExplicitComponent):

    def initialize(self):
        self.options.declare('vec_size', types=int, default=1,
                             desc="Total size of vector.")

    def setup(self):
        comm = self.comm
        rank = comm.rank

        size = self.options['vec_size']
        sizes, _ = evenly_distrib_idxs(comm.size, size)
        mysize = sizes[rank]

        # Distributed Input
        self.add_input('in_dist', np.ones(mysize, float), distributed=True)

        # Non-Distributed Input
        self.add_input('in_nd', np.ones(size, float))

        # Distributed Output
        self.add_output('out_dist', copy_shape='in_dist', distributed=True)

    def compute(self, inputs, outputs):
        x = inputs['in_dist']
        y = inputs['in_nd']

        # "Computationally Intensive" operation that we wish to parallelize.
        f_x = x**2 - 2.0*x + 4.0

        # This operation is repeated on all procs.
        f_y = y ** 0.5

        outputs['out_dist'] = f_x + np.sum(f_y)

size = 7

if MPI:
    comm = MPI.COMM_WORLD
    rank = comm.rank
    sizes, offsets = evenly_distrib_idxs(comm.size, size)
else:
    # When running without MPI, the entire variable is on one proc.
    rank = 0
    sizes = {rank : size}
    offsets = {rank : 0}

prob = om.Problem()
model = prob.model

# Create a distributed source for the distributed input.
ivc = om.IndepVarComp()
ivc.add_output('x_dist', np.zeros(sizes[rank]), distributed=True)
ivc.add_output('x_nd', np.zeros(size))

model.add_subsystem("indep", ivc)
model.add_subsystem("D1", MixedDistrib2(vec_size=size))
model.add_subsystem("D2", MixedDistrib2(vec_size=size))

model.connect('indep.x_dist', 'D1.in_dist')
model.connect('indep.x_nd', 'D1.in_nd')

# Distributed output to distributed input.
model.connect('D1.out_dist', 'D2.in_dist')

# Distributed output to non-distributed input.
# The Slicer is used to select all elements in D1.out_dist.
model.connect('D1.out_dist', 'D2.in_nd', src_indices=om.slicer[:])


prob.setup(force_alloc_complex=True)

# Set initial values of distributed variable.
x_dist_init = 3.0 + np.arange(size)[offsets[rank]:offsets[rank] + sizes[rank]]
prob.set_val('indep.x_dist', x_dist_init)

# Set initial values of non-distributed variable.
x_nd_init = 1.0 + 2.0*np.arange(size)
prob.set_val('indep.x_nd', x_nd_init)

prob.run_model()

# Values on each rank.
for var in ['D1.out_dist', 'D2.in_dist']:
    print(var, prob.get_val(var))
    
# Note: to get the value of a non-distributed input that is connected to a distributed output, you 
# need to set get_remote to True.
print('D2.in_nd', prob.get_val('D2.in_nd', get_remote=True))

In [ ]:
%%px
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(prob.get_val('D2.in_nd', get_remote=True), np.array([24.53604616, 29.53604616, 36.53604616, 45.53604616, 56.53604616, 69.53604616, 84.53604616]), 1e-6)